In [1]:
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt
import math
import cvzone
import numpy as np

In [2]:
video_path = 'video.mp4'
frames = list()

# Load the video
video = cv2.VideoCapture(video_path)

while True:
  # Read the next frame
  ret, frame = video.read()
  if not ret:
    break

  frame = cv2.resize(frame, (512, 512))
  frames.append(frame)

frames = np.array(frames)
frames.shape

(1200, 512, 512, 3)

In [3]:
model = YOLO("yolov8n.pt")

In [ ]:
# Load a model
bbox_raw = list()

for frame in frames:
  results = model(frame)
  for result in results:
    boxes = result.boxes
    for box in boxes:
      conf = box.conf[0].item()
      if conf >= 0.6:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2-x1, y2-y1
        cvzone.cornerRect(frame, (x1, y1, w, h))

  bbox_raw.append(frame)

bbox_raw = np.array(bbox_raw)
bbox_raw.shape

In [5]:
def findHomography(img1, img2):
    # Detect keypoints and descriptors in both images
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # Match features between the images
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # Filter matches using Lowe's ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    # Find homography using RANSAC
    if len(good) > 4:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        matchesMask = mask.ravel().tolist()
    else:
        M = None

    return M


In [6]:
homography_matrix = np.array([[-3.68932364e-04, -3.98732829e-03,  9.84008439e-01],
                     [-4.89824970e-05, -8.77735907e-04,  1.78061186e-01],
                     [-9.99893210e-07, -8.96856626e-06,  2.19072220e-03]])

In [7]:
warpPerspective_list = list()
for i in range(len(bbox_raw)):
  M = findHomography(frames[0], frames[i])
  if M is not None:
    warped_img = cv2.warpPerspective(bbox_raw[i], M, (bbox_raw[i].shape[1], bbox_raw[i].shape[0]))
  else:
    warped_img = bbox_raw[i]
  result = cv2.warpPerspective(warped_img, homography_matrix, (512, 512))
  result = np.rot90(np.rot90(np.rot90(result)))
  warpPerspective_list.append(result)

In [ ]:
warpPerspective_list = list()
for i in bbox_raw:
  result = cv2.warpPerspective(i, homography_matrix, (512, 512))
  result = np.rot90(np.rot90(np.rot90(result)))
  warpPerspective_list.append(result)

In [8]:
output = cv2.VideoWriter('output3.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 20, (512,512))
for i in range(len(warpPerspective_list)):
    output.write(warpPerspective_list[i])
output.release()
print("Video generated successfully!")

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Video generated successfully!


In [9]:
output = cv2.VideoWriter('output2.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 20, (512,512))
for i in range(len(bbox_raw)):
    output.write(bbox_raw[i])
output.release()
print("Video generated successfully!")

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Video generated successfully!
